In [82]:
# 항상 첫 번째 셀
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [83]:
def crawl_musinsa_goods(category_code="103", max_pages=1000):
    """
    무신사 API를 통해 상품 ID와 썸네일 URL을 수집합니다.
    :param category_code: 수집할 카테고리 코드 (예: 001)
    :param max_pages: 수집할 최대 페이지 수
    :return: 수집된 데이터 리스트
    """
    
    collected_data = []
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Referer": "https://www.musinsa.com/",
        "Accept": "application/json"
    }

    print(f"🚀 크롤링 시작: 카테고리 {category_code}, 최대 {max_pages} 페이지")

    # tqdm을 사용하여 진행률 바 생성
    pbar = tqdm(range(1, max_pages + 1), unit="page")
    
    for page in pbar:
        # 진행 상황 텍스트 업데이트
        pbar.set_description(f"수집 중... Page {page}")

        params = {
            "gf": "M",
            "sortCode": "SALE_ONE_YEAR_COUNT",
            "category": category_code,
            "size": 60,
            "testGroup": "",
            "caller": "CATEGORY",
            "page": page,
            "seen": 0,
            "seenAds": ""
        }
        
        url = "https://api.musinsa.com/api2/dp/v1/plp/goods"

        try:
            response = requests.get(url, headers=headers, params=params, timeout=10)
            
            if response.status_code == 200:
                json_data = response.json()
                goods_list = json_data.get('data', {}).get('list', [])
                
                if not goods_list:
                    pbar.write(f"🛑 [Page {page}] 더 이상 상품이 없습니다. 수집을 종료합니다.")
                    break

                current_items = 0
                for item in goods_list:
                    item_info = {
                        "goodsNo": item.get("goodsNo"),
                        "thumbnail": item.get("thumbnail"),
                        "goodsName": item.get("goodsName"),
                        "brandName": item.get("brandName")
                    }
                    collected_data.append(item_info)
                    current_items += 1
                
                # 우측에 수집된 총 개수 표시 업데이트
                pbar.set_postfix(total_collected=len(collected_data), last_count=current_items)

            else:
                pbar.write(f"⚠️ [Page {page}] 에러 발생: Status {response.status_code}")
                break

        except Exception as e:
            pbar.write(f"❌ [Page {page}] 요청 실패: {e}")
            break

        # 랜덤 딜레이
        time.sleep(random.uniform(0.2, 0.5))

    return collected_data

In [84]:
from db_client import RDSClient
# from db_client import RDSClient # 이미 import 되어 있다고 가정

# DB 클라이언트 초기화 (필요하다면)
db_client = RDSClient()

def insert_product_data_with_mapping(data_list):
    """
    딕셔너리 리스트 형태의 데이터를 DB에 삽입하기 전에 키를 매핑합니다.
    (goodsNo -> product_id, brandName -> brand, thumbnail -> img_url)
    """
    if not isinstance(data_list, list):
        print("❌ 입력된 데이터가 리스트 타입이 아닙니다.")
        return

    # 1. 삽입 쿼리 정의 (기존과 동일)
    # product_id, brand, img_url 컬럼에 삽입합니다.
    insert_query = """
    INSERT INTO product_shoes_img (product_id, brand, img_url)
    VALUES (:product_id, :brand, :img_url);
    """
    
    total_count = len(data_list)
    success_count = 0
    print(f"🔄 총 {total_count}개의 데이터를 삽입 시도합니다.")

    # 2. 리스트를 순회하며 각 딕셔너리(로우)를 매핑 후 삽입
    for item in data_list:
        
        # 🚨 핵심 수정 부분: 키 매핑 딕셔너리 생성
        try:
            mapped_params = {
                'product_id': item['goodsNo'],
                'brand': item['brandName'],
                'img_url': item['thumbnail']
            }
        except KeyError as e:
            print(f"⚠️ 데이터 딕셔너리에 필요한 키 ({e})가 없습니다. 해당 로우 건너뜀: {item}")
            continue

        # 3. 쿼리 실행
        # 매핑된 딕셔너리(mapped_params)를 파라미터로 사용합니다.
        row_count = db_client.execute(insert_query, mapped_params)
        
        if row_count == 1:
            success_count += 1
        else:
            # 기본 키 중복 등의 이유로 삽입 실패할 수 있습니다.
            print(f"⚠️ product_id {mapped_params['product_id']} 삽입 실패. (영향 받은 행: {row_count})")

    print(f"\n--- 작업 완료 ---")
    print(f"총 {total_count}개 중 {success_count}개의 행을 성공적으로 삽입했습니다.")


✅ DB Engine (Pool) 생성 완료


In [85]:
from curl_cffi import requests
import time
import random
import pandas as pd
from tqdm import tqdm  # 진행률 바 라이브러리 추가
from db_client import RDSClient  # 제공해 주신 모듈 import

# --- 실행부 ---
if __name__ == "__main__":
    # 테스트를 위해 페이지 수 설정 (예: 10페이지)
    target_category = "103"
    target_pages = 1000
    result_list = crawl_musinsa_goods(category_code=target_category, max_pages=target_pages) # 응답 리스트: {id, }
  

🚀 크롤링 시작: 카테고리 103, 최대 1000 페이지


수집 중... Page 386:  38%|███▊      | 385/1000 [02:54<04:38,  2.21page/s, last_count=31, total_collected=23071]   

🛑 [Page 386] 더 이상 상품이 없습니다. 수집을 종료합니다.


In [86]:
result_list

[{'goodsNo': 3976350,
  'thumbnail': 'https://image.msscdn.net/images/goods_img/20240321/3976350/3976350_17115804993997_500.jpg',
  'goodsName': '에어 포스 1 07 M - 화이트 / CW2288-111',
  'brandName': '나이키'},
 {'goodsNo': 3276429,
  'thumbnail': 'https://image.msscdn.net/images/goods_img/20230504/3276429/3276429_16995107122382_500.jpg',
  'goodsName': 'SHOWER - 808 (a) / BLACK',
  'brandName': '킨치'},
 {'goodsNo': 4791040,
  'thumbnail': 'https://image.msscdn.net/images/goods_img/20250214/4791040/4791040_17398378397153_500.jpg',
  'goodsName': '신세틱 스웨이드 클로그 HPCV4FF401',
  'brandName': '슈펜'},
 {'goodsNo': 1163169,
  'thumbnail': 'https://image.msscdn.net/images/goods_img/20190923/1163169/1163169_16896579905804_500.jpg',
  'goodsName': '삼바 OG - 화이트:블랙 / B75806',
  'brandName': '아디다스'},
 {'goodsNo': 3976349,
  'thumbnail': 'https://image.msscdn.net/images/goods_img/20240321/3976349/3976349_17115804555461_500.jpg',
  'goodsName': '에어 포스 1 07 M - 블랙 / CW2288-001',
  'brandName': '나이키'},
 {'goodsNo

In [87]:
# ⭐️ 최적화된 방식 (Batch Insert)
import os
from sqlalchemy import create_engine, text
def optimized_insert_product_data(data_list):
    # ... (데이터 매핑은 동일하게 수행하여 mapped_data_list를 만듭니다)
    
    # 1. 쿼리 정의 (기존과 동일)
    insert_query = """
    INSERT INTO product_shoes_img (product_id, brand, img_url)
    VALUES (:product_id, :brand, :img_url);
    """
    
    # 2. 모든 데이터를 매핑하여 하나의 리스트로 준비
    mapped_data_list = []
    for item in data_list:
        mapped_data_list.append({
            'product_id': item['goodsNo'],
            'brand': item['brandName'],
            'img_url': item['thumbnail']
        })
    # 중복 제거를 위한 딕셔너리 생성
    # (나중에 나오는 데이터가 이전 데이터를 덮어씌웁니다)
    unique_data = {}
    
    for item in data_list:
        p_id = item['goodsNo']  # 기준이 되는 ID
        unique_data[p_id] = {
            'product_id': p_id,
            'brand': item['brandName'],
            'img_url': item['thumbnail']
        }
    
    # 2. 중복이 제거된 값들만 리스트로 변환
    deduplicated_list = list(unique_data.values())
    
    # db_client 모듈을 가정하고, 임시로 connection을 직접 사용한다고 가정
    if not db_client.engine: return
    
    print("🔄 배치 삽입 (executemany) 시도...")
    try:
        with db_client.engine.connect() as connection:
            # text()로 쿼리 준비
            stmt = text(insert_query)
            
            # executemany로 한 번에 실행
            result = connection.execute(stmt, deduplicated_list)
            
            # 한 번의 커밋으로 모든 변경사항 반영
            connection.commit() 
            
            print(f"✅ 배치 삽입 성공! 총 {result.rowcount}개 행 삽입.")
    except Exception as e:
        print(f"❌ 배치 삽입 오류 발생: {e}")

# **이 방식은 수 분 내외 (수십 초 ~ 몇 분)로 작업을 완료할 수 있습니다.**

In [88]:
optimized_insert_product_data(result_list)

🔄 배치 삽입 (executemany) 시도...
✅ 배치 삽입 성공! 총 23071개 행 삽입.
